## Pandas 加速

大家好，今天我们来看pandas加速的相关小技巧，不知道大家在刚刚接触pandas的时候有没有听过如下的说法

>**pandas太慢了，运行要等半天。。。。

其实我想说的是，慢不是pandas的错，大家要知道pandas本身是在Numpy上建立起来的包，在很多情况下是支持向量化运算的，而且还有C的底层设计，所以我今天
主要想从几个方面和大家分享一下pandas加速的小技巧，与往常一样，文章分成四部分，本文结构如下：



1. 使用datetime类型来处理和时间序列有关的数据
2. 批量计算的技巧
3. 通过HDFStore存储数据节省时间
4. 源码，相关数据及GitHub地址


现在就让我们开始吧

## 1. 使用datetime类型来处理和时间序列有关的数据

首先这里我们使用的数据源是一个电力消耗情况的数据(energy_cost.csv)，非常贴近生活而且也是和时间息息相关的，用来做测试在合适不过了，这个csv文件大家可以在第四部分找到下载的地方哈

In [1]:
import os
#这两行仅仅是切换路径，方便我上传Github，大家不用理会,只要确认csv文件和py文件再一起就行啦
os.chdir("F:\\Python教程\\segmentfault\\pandas_share\\Pandas之旅_07 谁说pandas慢")

现在让我们看看数据大概长什么样子

In [2]:
import numpy as np
import pandas as pd
f"Using {pd.__name__},{pd.__version__}"

'Using pandas,0.23.0'

In [3]:
df = pd.read_csv('energy_cost.csv',sep=',')
df.head()

,date_time,energy_kwh
0,2001/1/13 0:00,0.586
1,2001/1/13 1:00,0.580
2,2001/1/13 2:00,0.572
3,2001/1/13 3:00,0.596
4,2001/1/13 4:00,0.592


现在我们看到初始数据的样子了，主要有date_time和energy_kwh这两列，来表示时间和消耗的电力，比较好理解，下面让我们来看一下数据类型

In [4]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [5]:
type(df.iat[0,0])

str

这里有个小问题，Pandas和NumPy有dtypes（数据类型）的概念。如果未指定参数，则date_time这一列的数据类型默认object，所以为了之后运算方便，我们可以把str类型的这一列转化为timestamp类型:

In [6]:
df['date_time'] = pd.to_datetime(df['date_time'])
df.dtypes

date_time     datetime64[ns]
energy_kwh           float64
dtype: object

先在大家可以发现我们通过用pd.to_datetime这个方法已经成功的把date_time这一列转化为了datetime64类型

In [7]:
df.head()

,date_time,energy_kwh
0,2001-01-13 00:00:00,0.586
1,2001-01-13 01:00:00,0.580
2,2001-01-13 02:00:00,0.572
3,2001-01-13 03:00:00,0.596
4,2001-01-13 04:00:00,0.592


现在再来看数据会发现已经和刚才不同了，我们先在想要关注一下时间，因为这一期的主题是加速，让我们查看一下刚才的操作耗时:

In [16]:
df = pd.read_csv('energy_cost.csv',sep=',')

In [17]:
%%timeit -n 500 
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

df['date_time'] = convert(df, 'date_time')

643 µs ± 284 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


这里考虑到数据量，这个速度其实很慢的，如果我们有100倍，1000倍的数据量呢？现在让我们添加format参数加速一下：

In [18]:
df = pd.read_csv('energy_cost.csv',sep=',')

In [19]:
%%timeit -n 500
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%Y/%m/%d %H:%M')

df['date_time']=convert_with_format(df, 'date_time')

481 µs ± 53.8 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


大家可以看到速度快了很多，具体的日期自定义相关方法，大家点击[这里](http://strftime.org/)查看

## 2. 批量计算的技巧

首先，我们假设根据用电的时间段不同，国家电力收费价目表如下：

| 类型 | 分/kwh | 时间段 |
| --- | --- | --- |
| Peak | 28 | 17:00 to 24:00 |
| Shoulder | 20 | 7:00 to 17:00 |
| Off-Peak | 12 | 0:00 to 7:00 |

## 3. 通过HDFStore存储数据节省时间

## 4. 源码，相关数据及GitHub地址

这一期为大家分享了一些pandas加速的实用技巧，希望可以帮到各位小伙伴，当然，类似的技巧还有很多，希望如果大家有更好的办法可以在我的文章底下留言哈

我把这一期的ipynb文件，py文件以及我们用到的energy_cost.csv放到了Github上，大家可以点击下面的链接来下载：
 - Github仓库地址： [https://github.com/yaozeliang/pandas_share](https://github.com/yaozeliang/pandas_share/tree/master/Pandas%E4%B9%8B%E6%97%85_04%20pandas%E8%B6%85%E5%AE%9E%E7%94%A8%E6%8A%80%E5%B7%A7)


希望大家能够继续支持我，完结，撒花